# EDA

## Import libs

In [ ]:
import sys
import os
from IPython.display import display

# Add src path
try:
    module_path = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', 'src'))
except NameError:
     module_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))

# import libs from load_libs.py
try:
    import load_libs
    from load_data import load_stroke_data
    print("Import successful !")
except ImportError as e:
    print(f"Error: {e}. Verify '../src'.")
    sys.exit("Missing packages")


## Loading Dataset

In [23]:
print("Loading data...")
df = load_stroke_data()
display(df.head())

Loading data...
Attempting to load file: healthcare-dataset-stroke-data.csv from dataset fedesoriano/stroke-prediction-dataset
Dataset loaded successfully!


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


## Dataset stats

In [32]:
# Missing values
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
if missing_percent.sum() > 0:
    print("Missing values (%)")
    display(missing_percent[missing_percent > 0])
else:
    print("No missing values.")

# Duplicate lines
duplicate_rows = df.duplicated().sum()
print(f"Duplicate lines : {duplicate_rows}")

# Unique values per column in %
print("\nUnique values per column (%)")
unique_counts = df.nunique().sort_values()
unique_counts = unique_counts/len(df) * 100
display(unique_counts)

Missing values (%)


bmi    3.933464
dtype: float64

Duplicate lines : 0

Unique values per column (%)


hypertension           0.039139
heart_disease          0.039139
ever_married           0.039139
Residence_type         0.039139
stroke                 0.039139
gender                 0.058708
smoking_status         0.078278
work_type              0.097847
age                    2.035225
bmi                    8.180039
avg_glucose_level     77.866928
id                   100.000000
dtype: float64